In [2]:
# =====================================================================
#  C3  – Monotonic-before-plateau Verification
# ---------------------------------------------------------------------
#  Claim (C3)
#  ----------
#  For every trajectory that *does* reach a numerical steady state
#  (match_type == "(1)"  OR  "(2)"), the phase-difference δ(t) is
#  **monotone within a small tolerance** on the entire time interval
#  preceding that steady state.
#
#  “Monotone within tolerance” means:
#      • increasing   →  Δδ ≥ −EPS  for every time-step
#      • decreasing   →  Δδ ≤  EPS  for every time-step
#    where  EPS = 1 × 10⁻⁶.
#
#  Input
#  -----
#  • ode_steady_analysis.xlsx   ← produced by the C1 script.
#    We keep rows whose  match_type ∈ {"(1)", "(2)"}.
#
#  Test procedure
#  --------------
#  1. Re-integrate the ODE (identical mesh & tolerances to C1):
#         T_MAX = 100,  DT = 0.001
#         TIME_EVAL = np.arange(0, T_MAX+DT, DT)   ← *fixed output grid*
#         rtol = 1e-6,  atol = 1e-9
#
#  2. Extract the pre-plateau segment:  t < steady_start_time.
#
#  3. Classify that segment as one of
#         { "inc",  "dec",  "non" }
#     using the EPS-based monotone test above.
#
#  Output
#  ------
#  • Counts of  {inc, dec, non, solver_fail}.
#      – *solver_fail* covers both integrator failure **and**
#        cases with NaN steady_start_time in the input file.
#  • Overall pass-rate  (= 1 − non / total).
#  • Any “non” rows written to
#        monotone_violations_preplateau.csv
#    for manual inspection.
#
#  Implementation notes
#  --------------------
#  • b is fixed at 1.0 inside  single_cell_ode(…).  This matches the
#    settings used when generating the input spreadsheet.
#  • The global  TIME_EVAL  grid is passed unmodified to  solve_ivp,
#    guaranteeing that every worker reports δ(t) on the *same* time
#    mesh — useful when comparing floating-point differences.
#  • Multiprocessing with  Pool(cpu_count())  speeds up the sweep but
#    does not affect numerical results.
# =====================================================================

import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from math import sin, cos, pi
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# ---------- numerical setup ------------------------------------------
T_MAX   = 100.0
DT      = 0.001
TIME_EVAL = np.arange(0.0, T_MAX + DT, DT)
EPS_MONO  = 1e-6                 # tolerance for monotone check

# ---------- single-cell ODE (b = 1) ----------------------------------
def single_cell_ode(t, y, p1, p2, a):
    θ, ψ = y
    b = 1.0
    s2 = sin(2 * (ψ - θ))
    s  = sin(ψ - θ)
    c  = cos(ψ - θ)
    denom = 2 * ((a * s) ** 2 + (b * c) ** 2) ** 1.5
    num   = (a * b) * (a ** 2 - b ** 2) * s2
    dθ_dt = -1.0 - p1 * num / denom
    dψ_dt = -p2 * s2
    return [dθ_dt, dψ_dt]

# ---------- helper: inc / dec / non ---------------------------------
def monotone_flag(arr, eps=EPS_MONO):
    diff = np.diff(arr)
    if np.all(diff >= -eps):
        return "inc"
    if np.all(diff <=  eps):
        return "dec"
    return "non"

# ---------- worker ---------------------------------------------------
def check(row_tuple):
    _, row = row_tuple
    δ0, p1, p2, a = row["delta0"], row["p1"], row["p2"], row["a"]
    t_plateau     = row["steady_start_time"]

    θ0 = pi / 3
    ψ0 = θ0 + δ0

    sol = solve_ivp(
        lambda t, y: single_cell_ode(t, y, p1, p2, a),
        (0.0, T_MAX), [θ0, ψ0],
        t_eval=TIME_EVAL, rtol=1e-6, atol=1e-9
    )
    if not sol.success or np.isnan(t_plateau):
        return {"monotone": "solver_fail"}

    δ = sol.y[1] - sol.y[0]
    pre_seg = δ[TIME_EVAL < t_plateau]
    return {"monotone": monotone_flag(pre_seg)}

# ---------- main -----------------------------------------------------
def main():
    df_raw = pd.read_excel("ode_steady_analysis.xlsx")

    # keep only converged trajectories (both classes)
    df = df_raw[df_raw["match_type"].isin(["(1)", "(2)"])].reset_index(drop=True)
    print("Trajectories to check (match_type 1 or 2):", len(df))  # 7 560 expected

    with Pool(cpu_count()) as pool:
        results = list(
            tqdm(pool.imap_unordered(check, df.iterrows()),
                 total=len(df), desc="Integrating")
        )

    res_df = pd.DataFrame(results)
    counts = res_df["monotone"].value_counts().reindex(
        ["inc", "dec", "non", "solver_fail"], fill_value=0
    )

    print("\nMonotone-flag counts (pre-plateau):")
    print(counts.to_string())

    pass_rate = 1.0 - counts["non"] / len(res_df)
    print(f"\nOverall pass rate: {100*pass_rate:.2f}%")

    if counts["non"] > 0:
        res_df[res_df["monotone"] == "non"].to_csv(
            "monotone_violations_preplateau.csv", index=False
        )
        print("Violations saved to monotone_violations_preplateau.csv")

if __name__ == "__main__":
    main()


Trajectories to check (match_type 1 or 2): 7560


Integrating: 100%|██████████| 7560/7560 [00:13<00:00, 546.46it/s]



Monotone-flag counts (pre-plateau):
monotone
inc            6368
dec            1192
non               0
solver_fail       0

Overall pass rate: 100.00%
